In [86]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import numpy 
import pandas as pd

In [87]:
# load model and tokenizer in English
roberta = "oliverguhr/german-sentiment-bert"

model = AutoModelForSequenceClassification.from_pretrained(roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)

labels = ['Positive', 'Negative', 'Neutral']

In [102]:
df = pd.read_csv('data/tweets_ats.csv')
tweets = df['renderedContent'].tolist()

In [89]:
tweet = "@WohlgensingerN @JuereVoBaern @twintapp @PostFinance Also ich kann schon noch unterscheiden zwischen minütig und 12 bis 30 Stunden! Für wie blöd hältst Du mich? Und ich weiss, dass die Bank- @twintapp s in ECHTZEIT übertragen. Die Rede hier ist und war aber von @PostFinance zu @PostFinance, die sich dem Problem nicht mal annimmt."

In [108]:
df.head()

,url,date,content,renderedContent,id,user,replyCount,retweetCount,likeCount,quoteCount,...,inReplyToTweetId,inReplyToUser,mentionedUsers,coordinates,place,hashtags,cashtags,Positive,Negative,Neutral
0,https://twitter.com/phdupont/status/1576138607...,2022-10-01 09:15:01+00:00,Habe diese Woche meine Fail-Kreditkarte bei @P...,Habe diese Woche meine Fail-Kreditkarte bei @P...,1576138607655800832,"{'username': 'phdupont', 'id': 141318124, 'dis...",0,0,0,0,...,NaN,NaN,"[{'username': 'PostFinance', 'id': 127884579, ...",NaN,NaN,NaN,NaN,1,2,3
1,https://twitter.com/AkosNitsch/status/15759600...,2022-09-30 21:25:29+00:00,@PostFinance Blocked this bullshit,@PostFinance Blocked this bullshit,1575960045539323905,"{'username': 'AkosNitsch', 'id': 8580457896161...",0,0,0,0,...,1.571726e+18,"{'username': 'PostFinance', 'id': 127884579, '...","[{'username': 'PostFinance', 'id': 127884579, ...",NaN,NaN,NaN,NaN,1,2,3
2,https://twitter.com/WohlgensingerN/status/1575...,2022-09-30 19:56:59+00:00,@pbruegger @JuereVoBaern @twintapp @PostFinanc...,@pbruegger @JuereVoBaern @twintapp @PostFinanc...,1575937774980513792,"{'username': 'WohlgensingerN', 'id': 103507013...",0,0,0,0,...,1.575937e+18,"{'username': 'pbruegger', 'id': 1910854244, 'd...","[{'username': 'pbruegger', 'id': 1910854244, '...",NaN,NaN,NaN,NaN,1,2,3
3,https://twitter.com/pbruegger/status/157593710...,2022-09-30 19:54:18+00:00,@WohlgensingerN @JuereVoBaern @twintapp @PostF...,@WohlgensingerN @JuereVoBaern @twintapp @PostF...,1575937100431896576,"{'username': 'pbruegger', 'id': 1910854244, 'd...",1,0,0,0,...,1.575935e+18,"{'username': 'WohlgensingerN', 'id': 103507013...","[{'username': 'WohlgensingerN', 'id': 10350701...",NaN,NaN,NaN,NaN,1,2,3
4,https://twitter.com/WohlgensingerN/status/1575...,2022-09-30 19:45:11+00:00,@pbruegger @JuereVoBaern @twintapp @PostFinanc...,@pbruegger @JuereVoBaern @twintapp @PostFinanc...,1575934803295186944,"{'username': 'WohlgensingerN', 'id': 103507013...",1,0,0,0,...,1.575648e+18,"{'username': 'pbruegger', 'id': 1910854244, 'd...","[{'username': 'pbruegger', 'id': 1910854244, '...",NaN,NaN,NaN,NaN,1,2,3


In [109]:
# precprcess tweet
tweet_words = []

df['Positive'] = 1
df['Negative'] = 2
df['Neutral'] = 3


for counter in tweets:
    
    for word in tweet.split(' '):
        if word.startswith('@') and len(word) > 1:
            word = '@user'
    
        elif word.startswith('http'):
            word = "http"
        tweet_words.append(word)

    tweet_proc = " ".join(tweet_words)

    encoded_tweet = tokenizer(tweet_proc, return_tensors='pt')

    output = model(**encoded_tweet)

    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    df['Positive'][counter] = scores[0]
    df['Negative'][counter] = scores[1]
    df['Neutral'][counter] = scores[2]

    

C:\Users\yanni\AppData\Local\Temp\ipykernel_31000\3213222058.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Positive'][counter] = scores[0]
C:\Users\yanni\AppData\Local\Temp\ipykernel_31000\3213222058.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Negative'][counter] = scores[1]
C:\Users\yanni\AppData\Local\Temp\ipykernel_31000\3213222058.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Neutral'][counter] = 

RuntimeError: The size of tensor a (527) must match the size of tensor b (512) at non-singleton dimension 1

In [91]:
tweet_proc

'Habe diese Woche meine Fail-Kreditkarte bei @user gekündigt und bin nun froh ausschliesslich mit der @user Cumulus-Kreditkarte unterwegs zu sein. Sie sammelt doppelt Punkte, funktionierte selbst in den USA einwandfrei und ist erst noch gratis. Ist doch ganz normal. @user Blocked this bullshit @user @user @user @user Ich meinte auf mein  "bei mir geht es SOFORT" dass es evtl auch bis zur nächsten minute gehe.\r\n\r\nNimm einfache aussagen nicht gleich als angriff o.ä. herrgott nochmals. @user @user @user @user Also ich kann schon noch unterscheiden zwischen minütig und 12 bis 30 Stunden! Für wie blöd hältst Du mich? Und ich weiss, dass die Bank- @user s in ECHTZEIT übertragen. Die Rede hier ist und war aber von @user zu @user die sich dem Problem nicht mal annimmt. @user @user @user @user Also bei mir geht das meist sofort 😅 (sgkb) evtl ist es nur das Gefühl und es ist minütig @user @user @user Also sgkb Twint innert Sekunden. @user Hallo @user Dank für dein Feedback betreffend der Vid

In [ ]:
# load model and tokenizer in English
roberta = "oliverguhr/german-sentiment-bert"

model = AutoModelForSequenceClassification.from_pretrained(roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)

labels = ['Positive', 'Negative', 'Neutral']

In [ ]:
# sentiment analysis
encoded_tweet = tokenizer(tweet_proc, return_tensors='pt')

In [ ]:
# output = model(encoded_tweet['input_ids'], encoded_tweet['attention_mask'])
output = model(**encoded_tweet)

scores = output[0][0].detach().numpy()
scores = softmax(scores)

for i in range(len(scores)):
    
    l = labels[i]
    s = scores[i]
    print(l,s)

Positive 0.0065376777
Negative 0.9933622
Neutral 0.00010025848
